In [ ]:
!pip install ultralytics
!pip install torch torchvision torchaudio
!pip install matplotlib.pyplot
!pip install Pillow

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("All Physical Devices:", physical_devices)
print("GPUs:", physical_devices)

print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version:", tf.sysconfig.get_build_info()["cudnn_version"])



In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import numpy as np


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Define the dataset path
data_dir = '/home/iambrink/Shark_Object_Detection/Shark_Classification_Data_V2'

# Custom Dataset class for loading images
class CustomDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Traverse through each class folder
        for label, class_name in enumerate(os.listdir(root)):
            class_folder = os.path.join(root, class_name)
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                self.images.append(img_path)
                self.labels.append(label)  # Use class index as label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')  # Load image and convert to RGB
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)  # Apply transformations

        return image, label

# Define transformations
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Crop to 224x224 with random scaling
    transforms.RandomHorizontalFlip(),  # Randomly flip images
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    transforms.RandomRotation(15),  # Random rotation
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize

])

# Load datasets using the custom dataset class
train_dataset = CustomDataset(root=os.path.join(data_dir, 'train'), transform=data_transforms)
valid_dataset = CustomDataset(root=os.path.join(data_dir, 'valid'), transform=data_transforms)
test_dataset = CustomDataset(root=os.path.join(data_dir, 'test'), transform=data_transforms)

# Create DataLoaders for each dataset
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
validloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Get dataset sizes and class names
dataset_sizes = {
    'train': len(train_dataset),
    'valid': len(valid_dataset),
    'test': len(test_dataset)
}
class_names = os.listdir(os.path.join(data_dir, 'train'))  # Get class names from training data

# Print dataset sizes and class names for verification
print(f"Dataset Sizes: {dataset_sizes}")
print(f"Class Names: {class_names}")


In [ ]:
# Load EfficientNetB0 model pre-trained on ImageNet
model = models.efficientnet_b0(weights='DEFAULT')  # Update this for newer torchvision versions

# Modify the classifier layer to match your 10 shark species
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 10)  # 10 shark species

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Define loss function (cross-entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=.005)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.3, patience=3, verbose=True)


In [ ]:
base_model = '/home/iambrink/Shark_Object_Detection/Shark_AI_Classification_Model_V2.pt'

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed_value=42):
    # Set seed for Python's random module
    random.seed(seed_value)

    # Set seed for NumPy's random number generator
    np.random.seed(seed_value)

    # Set seed for PyTorch (both CPU and GPU, if applicable)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  

    # Ensure deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [ ]:
# Training and Validation Loop
num_epochs = 500
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for batch_idx, (data, target) in enumerate(tqdm(trainloader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += accuracy_score(target.cpu(), output.argmax(dim=1).cpu())
    train_loss /= len(trainloader)
    train_acc /= len(trainloader)

    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(validloader)):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            val_acc += accuracy_score(target.cpu(), output.argmax(dim=1).cpu())
    val_loss /= len(validloader)
    val_acc /= len(validloader)

    # Update ReduceLROnPlateau scheduler based on validation loss
    scheduler.step(val_loss)

    print(f'Epoch: {epoch+1}/{num_epochs} \tTrain Loss: {train_loss:.4f} \tTrain Acc: {train_acc:.4f} \tVal Loss: {val_loss:.4f} \tVal Acc: {val_acc:.4f}')

    # Save the model if the validation accuracy improves
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Best model saved with Val Acc: {best_val_acc:.4f}')


In [ ]:
best_val_acc

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np

# Initialize empty lists to store true labels, predicted labels, and probabilities
y_true = []
y_pred = []
y_prob = []

# Iterate over the test set
for inputs, labels in testloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Make predictions without computing gradients
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        probs = torch.nn.functional.softmax(outputs, dim=1)

    # Append true labels, predicted labels, and probabilities
    y_true.extend(labels.cpu().numpy())
    y_pred.extend(preds.cpu().numpy())
    y_prob.extend(probs.cpu().numpy())

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

# Compute accuracy and misclassified examples
accuracy = accuracy_score(y_true, y_pred)
missed = (y_true != y_pred).sum()
print(f'Misclassified examples: {missed}')
print(f"Accuracy: {100 * accuracy:.2f}%")
print(f"          {y_true.size - missed}/{y_true.size}")

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Optionally, you can print the confusion matrix as well
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)

# Add labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Display the plot
plt.show()